In [25]:
import csv
import pandas as pd
import re

In [26]:
# Data from https://www.kaggle.com/datasets/rakkesharv/spotify-top-10000-streamed-songs
df = pd.read_csv("source/Spotify-Top-10000-Streamed-Songs.csv").dropna(subset=['Song Name']).reset_index(drop=True)

In [27]:

def words(title, with_parends):
	'''
	words('Street Spirit (Fade Out)', True) -> ['Street', 'Spirit', 'Fade', 'Out']
	words('Street Spirit (Fade Out)', False) -> ['Street', 'Spirit']
	'''
	title = title.strip()
	if not with_parends:
		title = re.sub(r'\([^)]*\)', '', title).strip()
	
	return [word.strip("()") for word in re.split(r'\s+', title)]

df['starts_with'] = df['Song Name'].apply(lambda x: words(x, False)[0].lower())
df['ends_with'] = df['Song Name'].apply(lambda x: words(x, False)[-1].lower())

df['starts_with_inc_parends'] = df['Song Name'].apply(lambda x: words(x, True)[0].lower())
df['ends_with_inc_parends'] = df['Song Name'].apply(lambda x: words(x, True)[-1].lower())

with open("spotify-top10000-labels.txt", "w") as f:
	f.write("\n".join(df['Song Name']))

labelled_edges = []

for i, src in df.iterrows():
	linking_words = set([src['ends_with'], src['ends_with_inc_parends']])
	targets = df[df['starts_with'].isin(linking_words) | df['starts_with_inc_parends'].isin(linking_words)]
	for j, tar in targets.iterrows():
		if src['Song Name'] == tar['Song Name']:
			continue
		labelled_edges.append(((i, src['Song Name']), (j, tar['Song Name'])))

with open("spotify-top10000-graph.txt", "w") as f:
	f.write(str(len(df['Song Name'])) + "\n")

	for (i, _), (j, _) in labelled_edges:
		f.write(f'{i} {j}\n')

with open("spotify-top10000-labelled-edges.txt", "w") as f:
	for (i, src_title), (j, tar_title) in labelled_edges:
		f.write(f"{src_title} -> {tar_title} ({i} -> {j})\n")

In [34]:
# Runs for a long time, haven't tested

# import subprocess

# with open("spotify-top10000-graph.txt", "r") as f:
# 	result = subprocess.run(['../lpath','DFBNB'], stdin=f, stdout=subprocess.PIPE, text=True)

# print(result.stdout)

Modifying the C code to spit out best paths as they are found, we get a length 653 path pretty quick.

1 5005 6862 901 6200 6937 288 1208 4618 2123 2918 5529 211 2471 9502 6985 7720 1676 2954 5231 623 1360 1815 2343 4682 2451 1543 183 23 4949 5419 6638 384 3028 261 207 7165 1277 8593 22 3598 313 98 10147 281 421 154 1981 4654 1472 330 1572 724 4749 2710 10099 1563 404 2370 7050 9744 357 1942 500 7862 269 1002 6436 172 249 608 717 6761 335 2387 3365 672 4113 8814 9709 9883 453 1580 1925 251 1135 3363 650 1586 3017 366 10325 217 487 3331 5114 10540 136 786 3856 5677 1214 854 3782 817 2739 3280 917 5448 1473 2114 3429 1218 4586 2053 2665 730 2261 3990 2259 510 3415 373 4780 8728 1522 716 1999 2920 7095 4807 2642 5207 406 2230 2653 1316 2076 4015 2456 7090 6179 7192 592 1035 6305 6422 528 573 2395 5160 2095 856 1252 5861 6835 1017 4215 5657 2550 64 422 4122 6588 72 105 5819 4683 6114 4967 236 1867 1521 3849 1709 8034 1104 2415 2965 2438 213 2860 2421 5007 1849 4124 434 818 1720 1155 511 411 2841 4658 6386 2526 4169 7897 2914 1101 2059 5689 8529 3368 3630 3646 4084 2264 3086 3413 1186 11030 3746 6234 10046 651 1236 4091 10330 10585 2628 3672 797 4963 8283 10113 2972 8613 6614 801 3161 491 3193 360 9 720 1195 600 76 4630 1743 1178 6074 1147 4341 4639 298 1323 850 3558 3566 3695 1414 43 1258 1498 21 9686 10335 5499 1254 4277 4623 201 3900 10255 7698 1792 590 4800 9113 1871 10370 8720 391 1787 3098 7146 1091 2290 10955 636 2656 1074 10078 1591 4867 29 1502 3876 6447 2889 6197 195 631 5206 8096 10306 10610 189 4625 496 3309 9871 486 1247 3248 2907 3291 5472 992 8035 502 3887 602 9461 10173 2141 2098 3110 1129 6198 4059 5241 118 2172 2405 67 227 1975 69 6538 3314 4191 4352 8714 5754 1892 2042 5285 5635 8821 9012 423 2986 2919 6274 1382 6920 9272 849 2378 1371 3401 2992 499 1407 3479 4218 10629 2892 2289 700 8213 1819 768 698 967 3283 5558 6975 2467 892 7327 4981 8069 597 10706 231 3167 2778 3222 127 1582 876 4419 6138 3501 7089 9476 1590 3482 3875 3273 7246 1798 2465 4164 5882 50 1780 2648 7457 736 1833 4497 2037 10027 5836 205 9127 7483 8399 3752 8588 3223 326 3225 7939 6736 6881 3265 1404 2406 9524 112 3125 5375 8943 3287 3722 3806 7080 7183 2532 4783 5495 1039 2371 10779 10107 544 5000 10935 1808 6854 10123 493 10389 4579 1679 3652 5688 7265 9778 7411 701 6324 2109 5128 5617 6003 4710 540 5925 1504 8586 580 3476 5644 9740 250 3745 702 2692 5008 2263 10631 3758 2297 2002 4289 4391 4514 2313 2388 272 3773 3343 1095 10835 284 2519 4319 1970 8328 4377 2810 7477 3761 509 166 5408 4599 5061 7510 4952 3516 5120 4851 6840 872 1071 4855 10703 658 1388 1057 2087 5672 9634 8186 2987 4975 7884 8302 556 844 1045 7973 9493 4946 928 4043 2488 6537 6886 5418 1545 7902 515 5874 1005 4781 4769 788 7547 10204 10022 7038 9257 9710 1888 7827 1241 8664 7835 7654 8224 10105 5242 9787 1431 3647 5013 1257 10184 1321 6837 9689 9269 3324 8789 8857 11031 7930 10490 8545 749 2935 4360 5216 2595 4956 3927 10583 8160 9321 903 332 6631 5314 5996 9713 10327 4281 20 8753 9427 10047 1228 4891 8483 3683 6737 7142 8832 620 1774 7387 3912 7900 6109 2491 8360 3204 4787 4225 5535 9053 9954 5848 8383 11011 5789 6057 7374 10989 3065 2637 10143 8974 9645 2999 7635 5652 7757 2436 6149 2827 5890 7597 10026 4879 4540 3457 9902 2118 276 5503 2101 1723 9256



In [35]:
path = "1 5005 6862 901 6200 6937 288 1208 4618 2123 2918 5529 211 2471 9502 6985 7720 1676 2954 5231 623 1360 1815 2343 4682 2451 1543 183 23 4949 5419 6638 384 3028 261 207 7165 1277 8593 22 3598 313 98 10147 281 421 154 1981 4654 1472 330 1572 724 4749 2710 10099 1563 404 2370 7050 9744 357 1942 500 7862 269 1002 6436 172 249 608 717 6761 335 2387 3365 672 4113 8814 9709 9883 453 1580 1925 251 1135 3363 650 1586 3017 366 10325 217 487 3331 5114 10540 136 786 3856 5677 1214 854 3782 817 2739 3280 917 5448 1473 2114 3429 1218 4586 2053 2665 730 2261 3990 2259 510 3415 373 4780 8728 1522 716 1999 2920 7095 4807 2642 5207 406 2230 2653 1316 2076 4015 2456 7090 6179 7192 592 1035 6305 6422 528 573 2395 5160 2095 856 1252 5861 6835 1017 4215 5657 2550 64 422 4122 6588 72 105 5819 4683 6114 4967 236 1867 1521 3849 1709 8034 1104 2415 2965 2438 213 2860 2421 5007 1849 4124 434 818 1720 1155 511 411 2841 4658 6386 2526 4169 7897 2914 1101 2059 5689 8529 3368 3630 3646 4084 2264 3086 3413 1186 11030 3746 6234 10046 651 1236 4091 10330 10585 2628 3672 797 4963 8283 10113 2972 8613 6614 801 3161 491 3193 360 9 720 1195 600 76 4630 1743 1178 6074 1147 4341 4639 298 1323 850 3558 3566 3695 1414 43 1258 1498 21 9686 10335 5499 1254 4277 4623 201 3900 10255 7698 1792 590 4800 9113 1871 10370 8720 391 1787 3098 7146 1091 2290 10955 636 2656 1074 10078 1591 4867 29 1502 3876 6447 2889 6197 195 631 5206 8096 10306 10610 189 4625 496 3309 9871 486 1247 3248 2907 3291 5472 992 8035 502 3887 602 9461 10173 2141 2098 3110 1129 6198 4059 5241 118 2172 2405 67 227 1975 69 6538 3314 4191 4352 8714 5754 1892 2042 5285 5635 8821 9012 423 2986 2919 6274 1382 6920 9272 849 2378 1371 3401 2992 499 1407 3479 4218 10629 2892 2289 700 8213 1819 768 698 967 3283 5558 6975 2467 892 7327 4981 8069 597 10706 231 3167 2778 3222 127 1582 876 4419 6138 3501 7089 9476 1590 3482 3875 3273 7246 1798 2465 4164 5882 50 1780 2648 7457 736 1833 4497 2037 10027 5836 205 9127 7483 8399 3752 8588 3223 326 3225 7939 6736 6881 3265 1404 2406 9524 112 3125 5375 8943 3287 3722 3806 7080 7183 2532 4783 5495 1039 2371 10779 10107 544 5000 10935 1808 6854 10123 493 10389 4579 1679 3652 5688 7265 9778 7411 701 6324 2109 5128 5617 6003 4710 540 5925 1504 8586 580 3476 5644 9740 250 3745 702 2692 5008 2263 10631 3758 2297 2002 4289 4391 4514 2313 2388 272 3773 3343 1095 10835 284 2519 4319 1970 8328 4377 2810 7477 3761 509 166 5408 4599 5061 7510 4952 3516 5120 4851 6840 872 1071 4855 10703 658 1388 1057 2087 5672 9634 8186 2987 4975 7884 8302 556 844 1045 7973 9493 4946 928 4043 2488 6537 6886 5418 1545 7902 515 5874 1005 4781 4769 788 7547 10204 10022 7038 9257 9710 1888 7827 1241 8664 7835 7654 8224 10105 5242 9787 1431 3647 5013 1257 10184 1321 6837 9689 9269 3324 8789 8857 11031 7930 10490 8545 749 2935 4360 5216 2595 4956 3927 10583 8160 9321 903 332 6631 5314 5996 9713 10327 4281 20 8753 9427 10047 1228 4891 8483 3683 6737 7142 8832 620 1774 7387 3912 7900 6109 2491 8360 3204 4787 4225 5535 9053 9954 5848 8383 11011 5789 6057 7374 10989 3065 2637 10143 8974 9645 2999 7635 5652 7757 2436 6149 2827 5890 7597 10026 4879 4540 3457 9902 2118 276 5503 2101 1723 9256".split()
path = map(int, path)

print('\n'.join([df.iloc[i]['Song Name'] for i in path]))

Lucid Dreams
Dreams Money Can Buy
Buy A Heart
Heart On Ice
ICE
Ice Cold
Cold Water
Water
Water (Drowning Pt. 2)
2 Be Loved (Am I Ready)
Ready
Ready Set Roll
Roll In Peace
peace
Peace Of Mind
Mind
MiNd Of MiNdd (Intro)
INTRO
Intro
Intro (Hate On Me)
Me and Your Mama
Mama
MAMA
Mama Said
Said N Done
Done for Me
Me Too
Too Good
good 4 u
u
u love u
U Deserve It
It Ain't Me
Me You
you broke me first
First Class
Class Fight
Fight Song
Song For No One
One Dance
Dance To This
This Is What You Came For
For The Night
Night Running
Running Up That Hill (A Deal With God)
God is a woman
Woman
Woman Like Me
Me On Whiskey
Whiskey On You
You Right
Right On
On Me
Me And My Broken Heart
Heart Of Glass (Live from the iHeart Music Festival)
Glass House
House Party
Party Girl
GIRL LIKE ME
Me & You Together Song
Song About You
You Was Right
Right Back
Back In Blood
Blood Walk
Walk It Talk It
It Was A Good Day
Day Before
Before You Go
Go Flex
Flex (Ooh, Ooh, Ooh)
Flex
Flex Up
Up
Up The Smoke
Smoke Break
Break